In [ ]:
using Symbolics

In [ ]:
using Einsum

In [ ]:
using LinearAlgebra

# Thermodynamically based derivation of material models in Julia

In [ ]:
import Pkg; Pkg.add("Plots")

In [ ]:
using Interact
using Plots

In [ ]:
α = 5

In [ ]:
α

In [ ]:
@variables u_N;
u_N

In [ ]:
𝒫 = @variables E_N S_N c_N K γ;

In [ ]:
@variables uᵖ_N ω_N;
ℱᵥ = @variables σᵖ_N Y_N;
ℰ = [uᵖ_N, ω_N]
n_ℰ = length(ℰ)
ℱᵥ

# Potential I: Helmholtz free energy

In [ ]:
ρψ = (1 - ω_N) * E_N * (u_N - uᵖ_N)^2 / 2

In [ ]:
Υ = [-1, -1];

In [ ]:
∂ρψ_∂ℰ = Symbolics.gradient(ρψ, ℰ)

In [ ]:
ℱ = Array(Υ .* ∂ρψ_∂ℰ)
ℱ #, typeof(ℱ)

In [ ]:
δᵢⱼ = Matrix(I, n_ℰ, n_ℰ);

In [ ]:
length(ℰ)

In [ ]:
@einsum ℱ[i] := δᵢⱼ[i, j] * Υ[j] * ∂ρψ_∂ℰ[j] 

In [ ]:
∂ℱ_∂ℰ = Symbolics.jacobian(ℱ, ℰ)

In [ ]:
Symbolics.sparsejacobian(ℱ, ℰ)

In [ ]:
get_Eps = Symbolics.build_function(E_N, E_N, conv=True );

In [ ]:
get_Eps(10)

In [ ]:
get_Sig_Eps = Symbolics.build_function.(ℱ', [ℰ, E_N, S_N, c_N, K, γ] );

In [ ]:
get_Sig_Eps([0.2 0.1 0.3], 1000., 1e-3, 1., 0.0, 0.0)

In [ ]:
get_Eps = lambdify.(ℰ, (ℰ,))

In [ ]:
get_Eps[1](0,1)

In [ ]:
get_Sig_Eps.([0.2 0.1 0.3], 1000., 1e-3, 1., 0.0, 0.0)

# Potential II: Threshold function

In [ ]:
@vars x
@vars y
@vars x_c
@vars a b nonnegative=True
@vars c positive=True
x, y, x_c, a, b, c

In [ ]:
@vars x_0 
@vars x_bar y_bar nonnegative=True
@vars m nonnegative=True

In [ ]:
f_lin_ = sqrt(y^2) - (y_bar - m * (x-x_0))

In [ ]:
f_ell_ = sqrt((x-x_0-x_c)^2/a^2 + y^2/b^2) - c
f_ell_

In [ ]:
∂f_ell_∂x = f_ell_.diff(x);
∂f_ell_∂y = f_ell_.diff(y);

In [ ]:
eq1 = Eq(subs(f_ell_, (x, x_bar), (y, 0)), 0)
eq2 = Eq(subs(f_ell_, (x, x_0), (y, y_bar)), 0)
eq3 = Eq(subs((-∂f_ell_∂x / ∂f_ell_∂y), (x, x_0), (y, y_bar)), -m)
eq3

In [ ]:
sol1, sol2, sol3, sol4 = sympy.solve((eq1, eq2, eq3),(a, b, x_c))

In [ ]:
sol2[2]

In [ ]:
abx_subs = sympy.solve([eq1 eq2 eq3],[a, b, x_c])[2]
a_, b_, x_c_ = abx_subs
subs_abx = Dict(a => a_, b => b_, x_c => x_c_)

In [ ]:
f_lin_c_ = subs(f_lin_, (x, x_c_ + x_0), (y, 0))
f_ell_abx_ = subs(f_ell_, subs_abx)
f_ell_abxc_ = subs(f_ell_abx_, (x, x_c_ + x_0), (y,0))
eq4 = Eq(f_ell_abxc_, f_lin_c_)

In [ ]:
c_ = solve(eq4, c)[1]

In [ ]:
f_ell_solved_ = subs(f_ell_, (a, a_), (b, b_), (x_c, x_c_), (c, c_))

In [ ]:
y_trans_ = subs( -y_bar / (x_c) * (x - x_0 - x_c), (x_c, x_c_))

In [ ]:
SymPy.StrictGreaterThan(a,b)

In [ ]:
f_cap_domain_ = sympy.StrictGreaterThan(
    sympy.sign(x_bar-x_0) * sympy.sign(-m) * (sympy.Abs(y) - y_trans_), 0
    )

In [ ]:
get_y_trans = lambdify(y_trans_, (x, x_bar, y_bar, m, x_0))

In [ ]:
get_y_trans.( [0.1, 0.2, 0.3], 1, 1, 0.1, 0 )

In [ ]:
@vars f_s f_t f_c f_c0;

In [ ]:
subs_tension = Dict(x_0 => 0, x_bar => f_t, y_bar => f_s)
subs_shear = Dict(y_bar => f_s, x_0 => 0)
subs_compression = Dict(x_0 => -f_c0, x_bar => -f_c,  y_bar => f_s-m*(-f_c0) )

In [ ]:
function foo(x)
    (x .<= 0) .* x + (x .> 0) .* log.(x)
end

In [ ]:
f_solved_ = sympy.Piecewise(
    (subs(f_ell_solved_, subs_tension), subs(f_cap_domain_,subs_tension)),
    (subs(f_ell_solved_, subs_compression), subs(f_cap_domain_, subs_compression)),
    (subs(f_lin_, subs_shear), True)
)
f_solved_

In [ ]:
get_f_solved = lambdify(f_solved_, (x, y, f_t, f_c, f_c0, f_s, m))

In [ ]:
get_f_solved.([0 1], [3.1, 3.3, 3.8], 3, 30, 22, 3, 0.1)

In [ ]:
x_range = LinRange(-35, 8, 300)
y_range = LinRange(-10, 10, 300)

In [ ]:
@time f_range = get_f_solved.(x_range', y_range, 3, 30, 10, 3, 0.1);

In [ ]:
using Plots

In [ ]:
@time surface(x_range, y_range, f_range)

In [ ]:
sympy.ccode(f_solved_)

In [ ]:
df_dx_ = sympy.diff(f_solved_, [x, y]);

In [ ]:
df_dx_a = Array([df_dx_[i] for i in [0, 1]]);

In [ ]:
get_df_dx = lambdify(df_dx_a, (x, y, f_t, f_c, f_c0, f_s, m))

In [ ]:
get_df_dx.(-36, -0, 3, 30, 22, 3, 0.1)

In [ ]:
get_df_dx.(x_range, y_range, 3, 30, 22, 3, 0.1)

In [ ]:
sympy.ccode(df_dx_a)